In [39]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import torch_directml
import json

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [3]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,4,5,6,7,8,9,imbalance_price
0,2021-02-19 23:00:00+00:00,26.46,32.38,671.75,47,0.965926,0.651899,861.119028,926.349178,983.808472,1019.720369,1036.559988,1061.210665,1080.227788,1093.639341,1113.818395,32.75000
1,2021-02-19 23:30:00+00:00,29.81,32.38,905.45,48,0.965926,0.651899,858.607841,926.349178,984.768943,1021.501948,1052.039144,1073.881656,1086.375645,1096.412648,1115.348849,62.50000
2,2021-02-21 00:00:00+00:00,7.67,17.34,781.05,1,1.000000,0.625411,856.444273,943.273486,966.508933,1012.477756,1017.698130,1050.847672,1067.065234,1080.375968,1102.706761,11.79015
3,2021-02-21 00:30:00+00:00,9.82,17.34,694.45,2,1.000000,0.625411,905.556134,941.699683,974.826050,1020.245426,1032.295819,1052.043334,1066.754321,1083.938818,1101.349134,12.29000
4,2021-02-21 01:00:00+00:00,10.40,14.25,847.35,3,0.965926,0.625411,905.556134,938.840650,961.032128,1031.312999,1040.546292,1055.273404,1072.689896,1086.803323,1102.929533,16.88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,85.584636,147.450968,186.180698,257.545157,312.000836,389.209655,461.471346,565.748439,66.01000
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,44.062739,80.500728,101.853409,152.946134,213.340381,285.856261,352.111989,494.824570,66.01000
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-31.030380,1.674412,35.557060,80.555516,138.624007,177.132223,250.514362,333.223882,66.01000
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-38.988480,19.495393,37.660809,72.199724,128.661011,157.102188,214.652927,345.416021,66.01000


In [33]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [47]:
import numpy as np
from sklearn.model_selection import train_test_split

def create_sequences(X, y, sequence_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i+sequence_length])
        y_seq.append(y[i+sequence_length])
    return np.array(X_seq), np.array(y_seq)

# Angenommen, df_bbidding1 ist Ihr DataFrame
X = df_bbidding1[["day_ahead_price"]].values
y = df_bbidding1["day_ahead_price"].shift(384).values

# Entfernen Sie NaN-Werte, die durch das Verschieben entstanden sind
X = X[384:]
y = y[384:]

# Erstellen Sie Sequenzen
sequence_length = 192
X_seq, y_seq = create_sequences(X, y, sequence_length)

# Train-Test-Split
X_train_val, X_test, y_train_val, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

# Train-Validation-Split
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=False)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (26693, 192, 1)
y_train shape: (26693,)
X_val shape: (6674, 192, 1)
y_val shape: (6674,)
X_test shape: (8342, 192, 1)
y_test shape: (8342,)


In [48]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

def to_tensor(data):
    if isinstance(data, pd.Series) or isinstance(data, pd.DataFrame):
        return torch.tensor(data.values, dtype=torch.float32)
    elif isinstance(data, np.ndarray):
        return torch.tensor(data, dtype=torch.float32)
    else:
        raise TypeError(f"Unerwarteter Datentyp: {type(data)}")

# Convert to PyTorch tensors
X_train_tensor = to_tensor(X_train)
y_train_tensor = to_tensor(y_train).view(-1, 1)
X_val_tensor = to_tensor(X_val)
y_val_tensor = to_tensor(y_val).view(-1, 1)
X_test_tensor = to_tensor(X_test)
y_test_tensor = to_tensor(y_test).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = 1 # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                # Number of LSTM layers
output_size = 1               # Output size
dropout = 0.1                 # Dropout rate
learning_rate = 0.0001        # Learning rate for optimizer
batch_size = 64               # Batch size
num_epochs = 500              # Maximum number of epochs
patience = 15                 # Patience for early stopping
rel_improvement_threshold = 0.0001  # Relative improvement threshold

In [49]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [50]:
X_train_tensor.shape

torch.Size([26693, 192, 1])

In [51]:
print("X_batch shape:", X_batch.shape)

X_batch shape: torch.Size([32, 192, 1])


In [52]:
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)


        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

# torch.save(model.state_dict(), 'LSTM_day_ahead_price.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Training Loss: 162.6016
Epoch [1/500], Validation Loss: 144.8236
Best model saved with validation loss: 144.8236
Epoch [2/500], Training Loss: 156.0805
Epoch [2/500], Validation Loss: 139.3788
Best model saved with validation loss: 139.3788
Epoch [3/500], Training Loss: 150.6053
Epoch [3/500], Validation Loss: 134.1400
Best model saved with validation loss: 134.1400
Epoch [4/500], Training Loss: 145.2497
Epoch [4/500], Validation Loss: 129.0101
Best model saved with validation loss: 129.0101
Epoch [5/500], Training Loss: 139.9528
Epoch [5/500], Validation Loss: 123.9583
Best model saved with validation loss: 123.9583
Epoch [6/500], Training Loss: 134.7040
Epoch [6/500], Validation Loss: 118.9878
Best model saved with validation loss: 118.9878
Epoch [7/500], Training Loss: 129.5093
Epoch [7/500], Validation Loss: 114.0946
Best model saved with validation loss: 114.0946
Epoch [8/500], Training Loss: 124.3741
Epoch [8/500], Validation Loss: 109.2876
Best model saved with va

In [53]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [54]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03T22:00:00+00:00,2024-10-04,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-03T22:30:00+00:00,2024-10-04,788,584,1148,1320,1007,565,923,797,487
2,2024-10-03T23:00:00+00:00,2024-10-04,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-03T23:30:00+00:00,2024-10-04,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04T00:00:00+00:00,2024-10-04,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
667,2024-10-21T19:30:00+00:00,2024-10-21,263,343,406,434,482,501,514,547,590
668,2024-10-21T20:00:00+00:00,2024-10-21,266,345,405,436,485,504,516,551,592
669,2024-10-21T20:30:00+00:00,2024-10-21,257,341,397,441,482,508,515,550,585
670,2024-10-21T21:00:00+00:00,2024-10-21,249,338,389,435,478,508,514,553,587


In [55]:
from datetime import datetime, timedelta
df_api_new = df_api_new.rename(columns={
    "timestamp": "datetime"
})
df_api_new["datetime"] = pd.to_datetime(df_api_new["datetime"])
min_date = df_api_new.datetime.min() - timedelta(minutes=30)
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
df_half_hourly = pd.DataFrame({"datetime": datetimes})
df_half_hourly["datetime"] = pd.to_datetime(df_half_hourly["datetime"])
df_api_new = pd.merge(df_half_hourly, df_api_new, left_on='datetime', right_on='datetime', how='outer')
df_api_new

,datetime,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1003,2024-10-21 19:30:00+00:00,2024-10-21,263.0,343.0,406.0,434.0,482.0,501.0,514.0,547.0,590.0
1004,2024-10-21 20:00:00+00:00,2024-10-21,266.0,345.0,405.0,436.0,485.0,504.0,516.0,551.0,592.0
1005,2024-10-21 20:30:00+00:00,2024-10-21,257.0,341.0,397.0,441.0,482.0,508.0,515.0,550.0,585.0
1006,2024-10-21 21:00:00+00:00,2024-10-21,249.0,338.0,389.0,435.0,478.0,508.0,514.0,553.0,587.0


In [56]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.day/7)
df_api_new_merged

,datetime,prediction_date,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.21,1485.30,2024-09-26 22:00:00+00:00,2024-09-26,47.0,68.929966,7.20,21.21,0.866025,-0.222521
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.46,1364.20,2024-09-26 22:30:00+00:00,2024-09-26,48.0,-28.480000,7.20,3.46,0.866025,-0.222521
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.15,1688.95,2024-09-26 23:00:00+00:00,2024-09-27,1.0,-19.020000,3.87,3.15,0.965926,-0.222521
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.68,1661.10,2024-09-26 23:30:00+00:00,2024-09-27,2.0,-18.000000,3.87,-3.68,0.965926,-0.222521
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.52,1824.60,2024-09-27 00:00:00+00:00,2024-09-27,3.0,56.450000,0.45,-0.52,1.000000,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2024-10-21 19:30:00+00:00,2024-10-21,263.0,343.0,406.0,434.0,482.0,501.0,514.0,547.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,1.000000
1010,2024-10-21 20:00:00+00:00,2024-10-21,266.0,345.0,405.0,436.0,485.0,504.0,516.0,551.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,1.000000
1011,2024-10-21 20:30:00+00:00,2024-10-21,257.0,341.0,397.0,441.0,482.0,508.0,515.0,550.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,1.000000
1012,2024-10-21 21:00:00+00:00,2024-10-21,249.0,338.0,389.0,435.0,478.0,508.0,514.0,553.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,1.000000


In [63]:
X = df_api_new_merged[["day_ahead_price"]].dropna().values

In [67]:
y = df_api_new_merged[["day_ahead_price"]].dropna().shift(384).values

In [65]:
X.shape

(726, 1)

In [69]:
# Entfernen Sie NaN-Werte, die durch das Verschieben entstanden sind
X = X[384:]
y = y[384:]

# Erstellen Sie Sequenzen
sequence_length = 192
X_seq, y_seq = create_sequences(X, y, sequence_length)

print("X_seq shape:", X_seq.shape)

X_seq shape: (150, 192, 1)


In [70]:
#convert to tensor
X_seq_tensor = to_tensor(X_seq)
y_seq_tensor = to_tensor(y_seq).view(-1, 1)

# test with model
model.eval()
with torch.no_grad():
    y_pred = model(X_seq_tensor.to(device)).cpu().numpy()

# Plot the results
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(y_seq)), y=y_seq.flatten(), mode='lines', name='True'))
fig.add_trace(go.Scatter(x=np.arange(len(y_seq)), y=y_pred.flatten(), mode='lines', name='Predicted'))
fig.update_layout(title='Day Ahead Price Forecasting', xaxis_title='Time Step', yaxis_title='Price')
fig.show()

In [72]:
#average error
mae = np.mean(np.abs(y_seq - y_pred))
mae

78.12024266153972

In [ ]:
df_api_new_merged